In [1]:
#Estatísticos
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

#Manipulação de Dados
import pandas as pd
import numpy as np
from numpy.linalg import LinAlgError

#Utilitários
import pickle
import itertools
from datetime import date, timedelta
from joblib import Parallel, delayed
import warnings

#Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

#Ajustes dos Gráficos
rcParams['figure.figsize'] = 18, 12

from Fase3_Aux_Func import (agrupamento_periodico,
                            ajuste_datas_historica,
                            conversao_array,
                            plot_serie,
                            plot_decomposicao,
                            suavizacao_simples,
                            suavizacao_exponencial,
                            holt_winters,
                            arma,
                            arima_sazonal,
                            analise_residuos,
                            check_stationarity,
                            grid_search_best_model,
                            grid_search_best_model_no_parallel)


# Suprimir todos os warnings
warnings.filterwarnings('ignore')

In [2]:
global local_origem 
global data_corte_treinamento

local_origem = "C:/Lab Transp/Dados Processados"
data_corte_treinamento = '2023-04-30'

### 1. Carregramento dos Dados

In [3]:
'''
LEMBRA DE MUDAR O NOME DO ARQUIVO PARA df_final_tratado DEPOIS DE AUTALIZAR O BD_BILHETAGEM E RODAR O DF TRATADO
'''
df_bilhetagem = pd.read_csv(local_origem + "/Dataframe Final.csv", sep=";", decimal="." )
df_bilhetagem['Volume_Passageiros (BU + VT + DIN)'] = df_bilhetagem['Volume_Passageiros (BU + VT + DIN)'].astype(int)
df_bilhetagem.head()

,Sub_Prefeitura,Zona,Ano,Mes,Volume_Passageiros (BU + VT + DIN),Qtde_Linhas
0,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,856,62
1,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2282,62
2,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2793,62
3,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2818,62
4,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2928,62


In [4]:
'''
Irá realizar os ajustes de datas necessários, periorizando por dia, além de quebrar em 2 arquivos
'''
df_subpref, df_zonas = agrupamento_periodico(df_bilhetagem)

In [5]:
#Criação das Listas
Sub_Prefeituras = np.unique(df_bilhetagem['Sub_Prefeitura'].astype(str).to_list())
Zonas = np.unique(df_bilhetagem['Zona'].astype(str).to_list())

### 2. Ajuste dos Modelos (Grid Search)

In [7]:
# Loop para encontrar o melhor modelo para cada 'Sub_Prefeitura' e 'Zona'
for sub_prefeitura in Sub_Prefeituras:
    print("Ajustando modelos para Sub-Prefeitura de " + sub_prefeitura)
    best_models_subpref = grid_search_best_model_no_parallel(df_subpref, 'Sub_Prefeitura', sub_prefeitura)
    with open(f'best_models_subpref_{sub_prefeitura}.pkl', 'wb') as f:
        pickle.dump(best_models_subpref, f)

print("Modelos treinados e salvos com sucesso.")        

Ajustando modelos para Sub-Prefeitura de ARICANDUVA-FORMOSA-CARRAO
O intevalo corresponde a 123  datas
Séries geradas para o modelo de séries temporais
Modelos treinados e salvos com sucesso.


In [ ]:
for zona in Zonas:
    print("Ajustando modelos para Zona " + zona)
    best_models_zona = grid_search_best_model(df_zonas, 'Zona', zona)
    with open(f'best_models_zona_{zona}.pkl', 'wb') as f:
        pickle.dump(best_models_zona, f)

print("Modelos treinados e salvos com sucesso.")